In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import os
import math
import matplotlib.pyplot as plt
import pandas_datareader as web
try:
    import yfinance as yf
except:
    !pip install yfinance
    import yfinance as yf
from datetime import date, timedelta, datetime
from pandas.io.json import json_normalize
import sqlite3
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
def get_BS_DATA(ticker):
    df_BS = yf.Ticker(ticker)
    return df_BS.balance_sheet

def get_CF_DATA(ticker):
    df_CF = yf.Ticker(ticker)
    return df_CF.cashflow

def get_PNL_DATA(ticker):
    df_PNL = yf.Ticker(ticker)
    return df_PNL.financials

In [ ]:
def get_fundamental_data():
    list_of_stocks = input("Please enter multiple scrips separated by comma: ")
    final_list = list_of_stocks.split(",")
    print(final_list)
    return final_list

In [ ]:
def generate_financial_dataset(ticker):
    get_BS_DATA(ticker).to_csv("DF_BS_DATA.csv")
    get_CF_DATA(ticker).to_csv("DF_CF_DATA.csv")
    get_PNL_DATA(ticker).to_csv("DF_PNL_DATA.csv")

### link for [yahoo finance](http://finance.yahoo.com/lookup)


In [ ]:
choice = "Y"
lst_tickers = []
while(choice=="Y"):
    str_tickers = input("Enter the ticker from yahoo finance: ")
    lst_tickers.append(str_tickers)
    choice_user = input("Continue Y/N: ")
    if(choice_user.upper()!="Y"):
        choice="N"
    else:
        choice="Y"

In [ ]:
def ratio(num,den):
    val=0.0
    try:
        val = round (num / den,4)
    except:
        print("!! Error for ",num," & ",den)
    finally:
        return val

def per_calc(y,y_1):
    val = 0.0
    try:
        val = round ((y_1/y)-1,4)
    except:
        print("!! Error for ",y," & ",y-1)
    finally:
        return val
    
def print_ms(var,msg):
    print(msg)
    print("\n")
    print(var)
    print("\n")

# Project Todo List
1. Create a duplicate copy of index columns called description.
2. Liquidity Ratios:
    * Current Ratio => Current Assets / Current Liabilities
    * Quick Ratio => (Current Assets - Inventory) / Current Liabilities
    * Absolute Liquidity => Cash + Marketable Securities + Net Receivables + Debtors
    * Cash Ratio => (Cash + Marketable Securities) / Current Liability

In [ ]:
def ratio_scoring_yoy_asc(num_row_lst,den_row_lst,name):
    # current assets change list
    lst_cr = [ratio(num_row_lst[0],den_row_lst[0]),\
              ratio(num_row_lst[1],den_row_lst[1]),\
              ratio(num_row_lst[2],den_row_lst[2]),\
              ratio(num_row_lst[3],den_row_lst[3])
             ]
    lst_cr_mv = [per_calc(lst_cr[0],lst_cr[1]),per_calc(lst_cr[1],lst_cr[2]),per_calc(lst_cr[2],lst_cr[3])]
    lst_points = [1 if x>0 else 0 for x in lst_cr_mv]
    print_ms(lst_cr,"-- Ratios for "+name+" --")
    print_ms(lst_cr_mv, "-- Ratio movement for "+name+" --")
    print_ms(lst_points, "-- Ratio Points for "+name+" --")
    return [lst_points,lst_cr_mv,lst_cr]

In [ ]:
import traceback

In [ ]:
def evaluate_score(df,df_pnl,df_cf):
    
    lst_aggregator = {}
    lst_aggregator_lr = {}
    lst_aggregator_tr = {}
    lst_aggregator_pr = {}
    lst_aggregator_frr = {}
    
    lst_curr_qr = []
    lst_curr_lr = []
    lst_curr_car = []
    lst_inv_tr = []
    lst_inv_rr = []
    lst_wc_tr = []
    lst_as_tr = []
    lst_cc_rt = []
    lst_cp_tr = []
    lst_er_margin = []
    lst_roce = []
    lst_eps = []
    lst_debt_equity = []
    lst_int_cvg = []
    lst_debt_covg = []
    
    try:
        # Liquidity Ratio
        # calculation for current ratio
        try:
            lst_curr_lr = ratio_scoring_yoy_asc(df.loc['Current Assets'],df.loc['Current Liabilities'],"Current Ratio")
        except:
            print("\n Exception Ocurred while calculating current ratio")
        finally:
            lst_curr_lr = [1,1,1]
            
        # calculation of quick ratio
        try:
            lst_curr_qr = ratio_scoring_yoy_asc(df.loc['Current Assets']-df.loc['Inventory'],df.loc['Current Liabilities'],"Quick Ratio")
        except:
            print("\n Exception Ocurred while calculating quick ratio")
        finally:
            lst_curr_qr = [1,1,1]

        # calculation of cash ratio
        try:
            lst_curr_car = ratio_scoring_yoy_asc(df.loc['Cash Cash Equivalents And Short Term Investments'],df.loc['Current Liabilities'],"Cash Ratio")
        except:
            print("\n Exception Ocurred while calculating quick ratio")
        finally:
            lst_curr_car = [1,1,1]

        # aggregate to the liquidity ratio list
        lst_aggregator_lr["Current_Ratio"] = lst_curr_lr 
        lst_aggregator_lr["Quick_Ratio"] = lst_curr_qr
        lst_aggregator_lr["Cash_Ratio"] = lst_curr_car

        # aggregate to the main aggregator ratio
        #     lst_aggregator.append("Quick_Ratio")
        #     lst_aggregator.append(lst_aggregator_lr)
        lst_aggregator["Quick_Ratio"] = lst_aggregator_lr
    except:
        print("Exception Ocurred in Liquidity Block")
        traceback.print_exc()
        
    try:
        #TURNOVER RATIOS
        # calculation for inventory tunrover ratio
        try:
            lst_inv_tr = ratio_scoring_yoy_asc(df_pnl.loc['Reconciled Cost Of Revenue'],df.loc['Inventory'],"Inventory Turnover")
        except:
            print("\n Exception Ocurred while calculating inventory turnover ratio")
        finally:
            lst_inv_tr = [1,1,1]

        # calculation for receivable turnover ratio
        try:
            lst_inv_rr = ratio_scoring_yoy_asc(df_pnl.loc['Operating Revenue'],df.loc['Accounts Receivable'],"Account Receivable Turnover")
        except:
            print("\n Exception Ocurred while calculating receivables turnover ratio")
        finally:
            lst_inv_rr = [1,1,1]

        # calculation for working capital turnover ratio
        try:
            lst_wc_tr = ratio_scoring_yoy_asc(df_pnl.loc['Total Revenue'],df.loc['Working Capital'],"Working Capital Turnover")
        except:
            print("\n Exception Ocurred while calculating working capital ratio")
        finally:
            lst_wc_tr = [1,1,1]
            
        # calculation for asset turnover ratio
        try:
            lst_as_tr = ratio_scoring_yoy_asc(df_pnl.loc['Operating Revenue'],df.loc['Total Assets'],"Asset Turnover")
        except:
            print("\n Exception Ocurred while calculating asset turnover ratio")
        finally:
            lst_as_tr = [1,1,1]

        # calculation for cash conversion ratio
        try:
            lst_cc_rt = ratio_scoring_yoy_asc(df_cf.loc['Operating Cash Flow'],df_pnl.loc['Normalized EBITDA'],"Cash conversion ratio")
        except:
            print("\n Exception Ocurred while calculating cash conversion ratio")
        finally:
            lst_cc_rt = [1,1,1]

        # calculation for capital turnover ratio
        try:
            lst_cp_tr = ratio_scoring_yoy_asc(df_pnl.loc['Total Revenue'],df.loc['Invested Capital'],"Cash conversion ratio")
        except:
            print("\n Exception Ocurred while calculating capital turnover ratio")
        finally:
            lst_cp_tr = [1,1,1]

        # aggregate to turnover ratio list
        lst_aggregator_tr["Inv_Turnover_Ratio"] = lst_inv_tr
        lst_aggregator_tr["Receivable_TR"] = lst_inv_rr
        lst_aggregator_tr["WC_TR"] = lst_wc_tr
        lst_aggregator_tr["Asset_TR"] = lst_as_tr
        lst_aggregator_tr["Cash_Conv_Ratio"] = lst_cc_rt
        lst_aggregator_tr["Capital_TR"] = lst_cp_tr

        #  aggregate to the main aggregator ratio
        lst_aggregator["Turnover_Ratios"] = lst_aggregator_tr
    
    except:
        print("Exeption ocurred in turnover Ratio")
        traceback.print_exc()
    
    try:
        # OPERATING PROFITABILITY RATIO
        # Earning Margin
        try:
            lst_er_margin = ratio_scoring_yoy_asc(df_pnl.loc['Net Income'],df_pnl.loc['Total Revenue'],"Earning Margin")
        except:
            print("\n Exception Ocurred while calculating earning margin")
        finally:
            lst_er_margin = [1,1,1]

        # return on capital employed
        try:
            lst_roce = ratio_scoring_yoy_asc(df_pnl.loc['EBIT'],df.loc['Invested Capital'],"Return on Capital Employed")
        except:
            print("\n Exception Ocurred while calculating earning roce")
        finally:
            lst_roce = [1,1,1]

        # earnings per share
        try:
            lst_eps = ratio_scoring_yoy_asc(df_pnl.loc['Net Income'],df_pnl.loc['Basic Average Shares'],"Earnings per share")
        except:
            print("\n Exception Ocurred while calculating EPS")
        finally:
            lst_eps = [1,1,1]

        # aggregate to turnover ratio list
        lst_aggregator_pr["Earning_Margin"] = lst_er_margin
        lst_aggregator_pr["ROCE"] = lst_roce
        lst_aggregator_pr["EPS"] = lst_eps

        #  aggregate to the main aggregator ratio
        lst_aggregator["Operating_Profitability_Ratios"] = lst_aggregator_pr
     
    except:
        print("Exception ocurred in profitability ratio")
        traceback.print_exc()
    
    try:
        # FINANCIAL RISK RATIOS
        # Debt to Equity Ratio
        try:
            lst_debt_equity = ratio_scoring_yoy_asc(df.loc['Long Term Debt'],df.loc['Stockholders Equity'],"Debt to Equity")
        except:
            print("\n Exception Ocurred while calculating debt to equity ratio")
        finally:
            lst_debt_equity = [1,1,1]
            

        # Interest Coverage Ratio
        try:
            lst_int_cvg = ratio_scoring_yoy_asc(df_pnl.loc['Normalized EBITDA'],df_pnl.loc['Interest Expense'],"Interest Coverage Ratio")
        except:
            print("\n Exception Ocurred while calculating interest coverage ratio")
        finally:
            lst_int_cvg = [1,1,1]

        # Debt Service Coverage Ratio
        try:
            lst_debt_covg = ratio_scoring_yoy_asc(df_pnl.loc['EBIT'],df_pnl.loc['Interest Expense'],"Debt Service Coverage Ratio")
        except:
            print("\n Exception Ocurred while calculating debt service coverage ratio")
        finally:
            lst_debt_covg = [1,1,1]
            
        # aggregate to turnover ratio list
        lst_aggregator_frr["Debt_Equity"] = lst_debt_equity
        lst_aggregator_frr["Int_Covering"] = lst_int_cvg
        lst_aggregator_frr["Debt_Convering"] = lst_debt_covg

        #  aggregate to the main aggregator ratio
        lst_aggregator["Financial_Risk_Ratio"] = lst_aggregator_frr
    
    except:
        print("Exception ocurred in Financial Risk ratio")
        traceback.print_exc()
    
    return lst_aggregator

# Turnover Ratio
* Inventory turnover ratio
* Receivable turnover ratio
* Capital ratio
* Asset turnover ratio
* Net working capital ratio
* Cash conversion ratio

# Operating Profitability Ratio Analysis
* Earning Margin
* Return on Capital Employed
* Return on Equity
* Earnings per share

# Financial Risk Ratios

* Debt Equity Ratio
* Interest Coverage Ratios
* Debt Service Coverage Ratio

# Stability Ratios

* Fixed Asset Ratio
* Current Assets To Fixed Assets
* Proprietary Ratio
* Fixed Interest Cover
* Fixed Dividend Ratio

In [ ]:
evaluation_results = {}
for ticker in lst_tickers:
    df = get_BS_DATA(ticker)
    df_pnl = get_PNL_DATA(ticker)
    df_cf = get_CF_DATA(ticker)
    evaluation_results[ticker] = evaluate_score(df,df_pnl,df_cf)
    
# iterate through all the elements of the evaluation result dictionary

In [ ]:
df_eval_data = pd.DataFrame(columns=['TICKER','LIQUIDITY','TURNOVER','PROFITABILITY','FINANCIAL'])
for tickers in evaluation_results:
    print(tickers)